In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

# Recognizing Recyclable materials
class SimpleEdgeAI:
    def __init__(self):
        self.classes = ['plastic', 'paper', 'glass', 'metal',]
        self.model = None

    def create_data(self, samples=200):
        """Create synthetic dataset"""
        X, y = [], []
        for class_id in range(4):
            for _ in range(samples):
                # Generate synthetic 64x64 images with class-specific patterns
                img = np.random.rand(64, 64, 3)
                if class_id == 0: img[:,:,2] *= 1.5      # Plastic - blue
                elif class_id == 1: img[:,:,0] *= 1.3    # Paper - red
                elif class_id == 2: img[:,:,1] *= 1.4    # Glass - green
                else: img = np.mean(img, axis=2, keepdims=True).repeat(3, axis=2)  # Metal - gray

                X.append(img)
                y.append(class_id)
        return np.array(X), np.array(y)

    def build_model(self):
        """Build lightweight CNN"""
        self.model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(64, 64, 3)),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(4, activation='softmax')
        ])

        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return self.model

    def train(self, X, y, epochs=5):
        """Train the model"""
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Training model...")
        self.model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, verbose=1)

        # Evaluate
        test_acc = self.model.evaluate(X_test, y_test, verbose=0)[1]
        print(f"Test Accuracy: {test_acc:.3f}")
        return X_test, y_test

    def convert_to_tflite(self):
        """Convert to TensorFlow Lite"""
        print("Converting to TensorFlow Lite...")
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()

        # Save model
        with open('recyclable_model.tflite', 'wb') as f:
            f.write(tflite_model)

        print(f"TFLite model saved (Size: {len(tflite_model)/1024:.1f} KB)")
        return tflite_model

    def test_tflite_inference(self, X_test, y_test):
        """Test TensorFlow Lite model"""
        print("Testing TFLite inference...")

        # Load TFLite model
        interpreter = tf.lite.Interpreter(model_path='recyclable_model.tflite')
        interpreter.allocate_tensors()

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        predictions = []
        times = []

        # Test on 50 samples
        for i in range(min(50, len(X_test))):
            input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)

            start_time = time.time()
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            end_time = time.time()

            predictions.append(np.argmax(output))
            times.append(end_time - start_time)

        # Calculate metrics
        tflite_accuracy = accuracy_score(y_test[:len(predictions)], predictions)
        avg_time = np.mean(times) * 1000  # Convert to milliseconds

        print(f"TFLite Accuracy: {tflite_accuracy:.3f}")
        print(f"Average Inference Time: {avg_time:.2f} ms")

        return tflite_accuracy, avg_time

# Main execution
def main():
    print("=== Simple Edge AI Recyclable Classifier ===\n")

    # Initialize
    ai = SimpleEdgeAI()

    # Create dataset
    print("1. Creating synthetic dataset...")
    X, y = ai.create_data(samples=200)
    print(f"Dataset created: {X.shape[0]} samples, {X.shape[1:]} image size")

    # Build and train model
    print("\n2. Building and training model...")
    model = ai.build_model()
    print(f"Model parameters: {model.count_params():,}")

    X_test, y_test = ai.train(X, y, epochs=5)

    # Convert to TensorFlow Lite
    print("\n3. Converting to TensorFlow Lite...")
    tflite_model = ai.convert_to_tflite()

    # Test TFLite model
    print("\n4. Testing TensorFlow Lite model...")
    tflite_acc, inference_time = ai.test_tflite_inference(X_test, y_test)

    # Summary
    print("\n=== EDGE AI BENEFITS ===")
    print(f"✓ Fast Inference: {inference_time:.2f} ms per image")
    print(f"✓ Small Model: {len(tflite_model)/1024:.1f} KB")
    print(f"✓ Good Accuracy: {tflite_acc:.1%}")
    print(f"✓ Offline Capable: No internet required")
    print(f"✓ Privacy Preserving: Data stays on device")
    print(f"✓ Real-time Ready: <50ms inference")

    print("\n=== DEPLOYMENT STEPS ===")
    print("1. Install: pip install tflite-runtime")
    print("2. Copy 'recyclable_model.tflite' to Raspberry Pi")
    print("3. Use camera to capture images")
    print("4. Preprocess: resize to 64x64, normalize")
    print("5. Run inference using TFLite interpreter")
    print("6. Display classification results")

if __name__ == "__main__":
    main()

=== Simple Edge AI Recyclable Classifier ===

1. Creating synthetic dataset...
Dataset created: 800 samples, (64, 64, 3) image size

2. Building and training model...
Model parameters: 28,228
Training model...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.3138 - loss: 1.3483 - val_accuracy: 0.5625 - val_loss: 1.1571
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.5532 - loss: 1.0997 - val_accuracy: 1.0000 - val_loss: 0.7253
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 1.0000 - loss: 0.6357 - val_accuracy: 1.0000 - val_loss: 0.2566
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 1.0000 - loss: 0.2012 - val_accuracy: 1.0000 - val_loss: 0.0516
Epoch 5/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 156ms/step - accuracy: 1.0000 - loss: 0.0460 - val_accuracy: 1.0000 - val_loss: 0.0141
Test Accuracy: 1.000

3. Converting to TensorFlow Lite...
Converting to TensorFlow Lite...
Saved artifact at '/tmp/tmp2nlvrhf9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
